In [1]:
single_process_path = 'C:\\Users\\schle\Documents\\Incubator Data\\'

# settings that define the files that will be searched for in the load folder above.
# processes indicated below will be run on all such files found. 
subject = ['AO001R'] # ['AO001R', 'AO008R']
subject_line = ['--',':']

angle = ['temporal', 'nasal', 'superior', 'inferior']
angle_marker = ['o','x','^','s']

eccentricity = ['1.5', '3.5', '4', '4.5' '10']

conetype = ['all','L','M','S','unclassed'] #  ['all','L','M','S','unclassed']
conetype_color = ['y','r','g','b','w']

coord_unit = 'pixels' # units of the coordinate data loaded in - any string you want

#visualize_intracone_dist_hist = [['L'],['M'],['S']]

visualize_intracone_dist_hist = [['L','1.5'],['M','1.5'],['S','1.5'],['all','1.5']]


import numpy as np
import matplotlib.pyplot as plt
from scipy import spatial
import os
import h5py
import importlib
import glob
import yaml
import sys

import mosaic_topog.utilities as util
import mosaic_topog.flsyst as flsyst
import mosaic_topog.show as show
import mosaic_topog.calc as calc
import mosaic_topog.singleMosaicProcesses as smp

category = ['subject', 'angle', 'eccentricity', 'conetype', 'conetype_color']

category_data = {}
for cat in category:
    category_data[cat] = locals()[cat]

user_param_keys = [
              'subject',
              'subject_line',
              'angle',
              'angle_marker',
              'eccentricity',
              'conetype',
              'conetype_color',
              'coord_unit',
              'category_data'
             ]

wait_list = [
            'mosaic',
            'fl_name',
            'index',
            ]

user_param = dict.fromkeys(user_param_keys)

# most of these parameters have been set directly by the user, 
# a few need to be interpreted from the filesystem, however. 
for p in user_param:
    user_param[p] = locals()[p]

In [2]:
importlib.reload(flsyst)
importlib.reload(smp)
importlib.reload(calc)
importlib.reload(show)

<module 'mosaic_topog.show' from 'c:\\users\\schle\\documents\\github\\incubator-2022-geometry-of-color\\mosaic_topog\\mosaic_topog\\show.py'>

In [3]:
# view intracone dist hists by subject
mosaic, fl_name, index = flsyst.getFilesByDataGroup(single_process_path, user_param, '.hdf5')


found 35 files



In [4]:
for p in wait_list:
    user_param[p] = locals()[p]

In [5]:
def getFromCat(var, category):
    for cat in category.keys():
        if var in category[cat]:
            return cat, category[cat].index(var)
    if not var:
        return []

In [6]:
def plotSettingsFromIndex(fl_inds_to_get,index,plot_settings):
    settings = {}
    settings['line'] = []
    settings['marker'] = []
    settings['color'] = []
    settings['markersize'] = []

    conetype_color = plot_settings['conetype_color']
    angle_marker = plot_settings['angle_marker']
    subject_line = plot_settings['subject_line']
    ecc_size = plot_settings['ecc_size']

    for fl in fl_inds_to_get:
        settings['color'].append(conetype_color[index['conetype'][fl]])
        settings['marker'].append(angle_marker[index['angle'][fl]])
        settings['line'].append(subject_line[index['subject'][fl]])
        settings['markersize'].append(ecc_size[index['eccentricity'][fl]])

    return settings

In [7]:
def visualizeIntraconeDistHist(variables, user_param):
    # unpack critical variables 
    category = user_param['category_data']
    coord_unit = user_param['coord_unit']
    fl_name = user_param['fl_name']
    index = user_param['index']
    
    plot_settings = {}
    plot_settings['subject_line'] = user_param['subject_line']
    plot_settings['angle_marker'] = user_param['angle_marker']
    plot_settings['conetype_color'] = user_param['conetype_color']
    plot_settings['ecc_size'] = user_param['eccentricity']

    for var_set in variables:
        #find the files to read the data from
        fl_inds_to_get = {}
        id_str = 'intracone dist hists: '
        for ind, var in enumerate(var_set):
            try:
                cat, ind = getFromCat(var, category) 
            except:
                print('requested var is not specified by any category')
                
            if ind == 0:
                id_str = id_str + var
            else: 
                id_str = id_str + " + " + var
                
            var_inds = np.nonzero(index[cat]==ind)[0]
            
            if not cat in fl_inds_to_get.keys():
                fl_inds_to_get[cat] = var_inds
            else: 
                fl_inds_to_get[cat] = np.append(fl_inds_to_get[cat], var_inds)
        
        temp = []
        for key in fl_inds_to_get.keys():
            if len(temp) == 0:
                temp = fl_inds_to_get[key]
            else:
                temp = np.intersect1d(temp,fl_inds_to_get[key])

        fl_inds_to_get = temp
        
        settings = plotSettingsFromIndex(fl_inds_to_get, index, plot_settings)
            
        hists = []
        bin_edges = []
        for fl in fl_inds_to_get:
            with h5py.File(fl_name[fl], 'r') as file:
                bin_edges.append(file['intracone_dist']['bin_edge'][()])
                hists.append(file['intracone_dist']['hist'][()])
            
        fig, ax = plt.subplots(1, 1, figsize = [10,10])
        
        for ind, hist in enumerate(hists):
            if hist.shape[0]>2:
                x = bin_edges[ind][1:]/2
                show.line(bin_edges[ind][1:], hists[ind], id_str, ax=ax, 
                          plot_col=settings['color'][ind], 
                          linestyle=settings['line'][ind], 
                          marker=settings['marker'][ind],
                          markersize= 4) #settings['markersize'][ind])
            else:
                print("warning: histogram for " + fl_name[fl] + " has 2 or fewer bins")

In [ ]:
visualizeIntraconeDistHist(visualize_intracone_dist_hist, user_param)

In [ ]:
'L' in conetype